# Project 3:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [1]:
# PREPARE REQUIRED LIBRARIES
import scrapy
from scrapy.selector import Selector
import requests
import pandas as pd
import numpy as np
from lxml import html



## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [2]:
# SCRAPE WIKIPEDI<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: A FOR LARGEST US CITIES 
#(NON-EXHAUSTIVE LIST)
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
r = requests.get(url)
tree = html.fromstring(r.content)
#This will create a list of cities:
cities = tree.xpath('//i/a/@title')

In [3]:
cities

['New York City',
 'Los Angeles',
 'Chicago',
 'Houston',
 'Philadelphia',
 'Jacksonville, Florida',
 'Charlotte, North Carolina',
 'Seattle',
 'Detroit',
 'Portland, Oregon',
 'Las Vegas',
 'Louisville, Kentucky',
 'Baltimore',
 'Milwaukee',
 'Albuquerque, New Mexico',
 'Kansas City, Missouri',
 'Virginia Beach',
 'Omaha, Nebraska',
 'Minneapolis',
 'New Orleans',
 'Wichita, Kansas',
 'Anchorage, Alaska',
 'Newark, New Jersey',
 'Birmingham, Alabama',
 'Sioux Falls, South Dakota',
 'Bridgeport, Connecticut',
 'Charleston, South Carolina',
 'Fargo, North Dakota',
 'Manchester, New Hampshire',
 'Billings, Montana',
 'Municipalities of Puerto Rico']

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [4]:
# ONLY RETAIN PROPERLY FORMED CITIES WITH FILTERING FUNCTION
cities_ascii = [c.encode('ascii') for c in cities]

In [5]:
print len(cities_ascii),len(cities)

31 31


## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [6]:
#pick up LA
def craiglist_price(city):
    url_city ='https://' + city + '.craigslist.org/search/sss?query=rv&sort=rel'
    r_city = requests.get(url_city)
    tree = html.fromstring(r_city.content)
    prices = tree.xpath('//span[@class="result-price"]/text()')
    return prices

craiglist_price('losangeles')

['$3650',
 '$3650',
 '$25',
 '$1',
 '$250',
 '$20',
 '$20',
 '$40',
 '$40',
 '$1892',
 '$14900',
 '$14900',
 '$3500',
 '$3500',
 '$1',
 '$1',
 '$40',
 '$40',
 '$49',
 '$49',
 '$74995',
 '$74995',
 '$79995',
 '$79995',
 '$84995',
 '$84995',
 '$14000',
 '$14000',
 '$14000',
 '$14000',
 '$7000',
 '$7000',
 '$200',
 '$200',
 '$14000',
 '$14000',
 '$145',
 '$145',
 '$1',
 '$25',
 '$25',
 '$29000',
 '$29000',
 '$25000',
 '$25000',
 '$225000',
 '$225000',
 '$80',
 '$80',
 '$5500',
 '$5500',
 '$5200',
 '$5200',
 '$100',
 '$100',
 '$11000',
 '$11000',
 '$11000',
 '$11000',
 '$1',
 '$1',
 '$9000',
 '$9000',
 '$375',
 '$375',
 '$29000',
 '$29000',
 '$4500',
 '$4500',
 '$4900',
 '$4900',
 '$3500',
 '$3500',
 '$5',
 '$5',
 '$9995',
 '$9995',
 '$85500',
 '$85500',
 '$600',
 '$20950',
 '$20950',
 '$4900',
 '$4900',
 '$100',
 '$100',
 '$95000',
 '$95000',
 '$2000',
 '$2000',
 '$5500',
 '$5500',
 '$6599',
 '$6599',
 '$2',
 '$2',
 '$12750',
 '$12750',
 '$100',
 '$100',
 '$25000',
 '$25000',
 '$60',
 '$6


## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [7]:
new_cities = []
for city in cities:
    new_cities.append(city.split(',')[0].replace(' ','').lower())

In [8]:
#replace newyork city with newyork
new_cities[0] = 'newyork'
#there is no page for virginia beach,bridgeport and municipalitiesofpuertorico, drop this line
new_cities.remove('virginiabeach')
new_cities.remove('bridgeport')
new_cities.remove('municipalitiesofpuertorico')
#no newark, there is new jersey
new_cities = ['newjersey' if c == 'newark' else c for c in new_cities]

In [9]:
new_cities

['newyork',
 'losangeles',
 'chicago',
 'houston',
 'philadelphia',
 'jacksonville',
 'charlotte',
 'seattle',
 'detroit',
 'portland',
 'lasvegas',
 'louisville',
 'baltimore',
 'milwaukee',
 'albuquerque',
 'kansascity',
 'omaha',
 'minneapolis',
 'neworleans',
 'wichita',
 'anchorage',
 'newjersey',
 'birmingham',
 'siouxfalls',
 'charleston',
 'fargo',
 'manchester',
 'billings']

In [10]:
cities_price = {}
for city in new_cities:
    cities_price[city] = [int(item[1:]) for item in craiglist_price(city)]

In [11]:
cities_price

{'albuquerque': [4500,
  4500,
  75,
  75,
  29000,
  29000,
  3500,
  3500,
  29000,
  29000,
  15500,
  15500,
  28900,
  28900,
  8900,
  8900,
  1550,
  1550,
  1,
  1,
  329,
  329,
  125,
  125,
  25000,
  25000,
  38500,
  38500,
  23500,
  23500,
  17500,
  17500,
  50,
  50,
  50,
  50,
  27,
  27,
  100,
  280,
  280,
  51500,
  51500,
  6000,
  6000,
  5500,
  5500,
  19800,
  19800,
  250,
  250,
  5900,
  5900,
  400,
  9500,
  9500,
  1,
  29500,
  29500,
  21500,
  21500,
  14900,
  14900,
  7850,
  7850,
  29750,
  17000,
  17000,
  50,
  50,
  8200,
  8200,
  6000,
  6000,
  6000,
  6000,
  40000,
  40000,
  89000,
  89000,
  38000,
  38000,
  190,
  54900,
  54900,
  6750,
  7000,
  22000,
  22000,
  15500,
  15500,
  46900,
  46900,
  8998,
  8998,
  11998,
  11998,
  12998,
  12998,
  4525,
  4525,
  75,
  75,
  28998,
  28998,
  33500,
  33500,
  55000,
  50000,
  50000,
  30,
  30,
  3500,
  3500,
  4800,
  4800,
  650,
  2825,
  2825,
  3350,
  3350,
  95,
  95,



## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [12]:
import numpy as np
city_mean_price= {}
for city,price in cities_price.items(): 
    city_mean_price[city] = np.mean(price)

/Users/yangliu/anaconda2/envs/dsi/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/yangliu/anaconda2/envs/dsi/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
city_mean_price

{'albuquerque': 16828.350574712644,
 'anchorage': 15389.598901098901,
 'baltimore': 10391.038647342995,
 'billings': 23027.67724867725,
 'birmingham': nan,
 'charleston': 11663.700000000001,
 'charlotte': 13922.648648648648,
 'chicago': 9713.5561224489793,
 'detroit': 10604.941747572815,
 'fargo': 17750.64814814815,
 'houston': 14895.252525252525,
 'jacksonville': 21718.755319148935,
 'kansascity': 10506.242268041236,
 'lasvegas': 15205.082872928177,
 'losangeles': 16524.075376884422,
 'louisville': 16808.742718446603,
 'manchester': 146.5,
 'milwaukee': 18520.732673267328,
 'minneapolis': 19676.19248826291,
 'newjersey': 16836.275000000001,
 'neworleans': 20057.545454545456,
 'newyork': 7630.6373056994817,
 'omaha': 26101.091787439615,
 'philadelphia': 12792.063829787234,
 'portland': 23124.994011976047,
 'seattle': 14986.740157480315,
 'siouxfalls': 28729.615763546797,
 'wichita': 11777.005235602095}


## 4. Run your scraping process, and save your results to a CSV file.

In [14]:
import csv
import urllib2
from bs4 import BeautifulSoup

with open('craiglists.csv', 'wb') as f:
    writer = csv.writer(f)


## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

In [15]:
#there is price difference between RVs in different State


### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

In [17]:
prices = []
for key,values in city_mean_price.items():
    prices.append(values)

In [21]:
#remove outlier
prices = [p for p in prices if p > 1000]

In [23]:
max(prices) - min(prices)

21098.978457847315

In [ ]:
# there is 21098 price difference between the max and min, so it makes sense

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?


## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).


## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._